##### import lIbraries

In [30]:
import os
from tweepy import OAuthHandler as OAuth, API, Cursor
import pandas as pd

##### load dotenv to expose api keys to app

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

##### add api keys as specified in environment variable

In [3]:
CONSUMER_KEY = 'API_key'
CONSUMER_SECRET = 'API_secret_key'
ACCESS_TOKEN = 'Access_token'
ACCESS_SECRET = 'Access_token_secret'

##### get keys from environment

In [4]:
CONSUMER_KEY = os.environ.get(CONSUMER_KEY)
CONSUMER_SECRET = os.environ.get(CONSUMER_SECRET)
ACCESS_TOKEN = os.environ.get(ACCESS_TOKEN)
ACCESS_SECRET = os.environ.get(ACCESS_SECRET)

##### set consumer keys

In [5]:
auth = OAuth(CONSUMER_KEY, CONSUMER_SECRET)

##### set access token

In [6]:
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

##### connect to twitter API

In [7]:
api = API(auth, wait_on_rate_limit=True)

##### test connection

In [8]:
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text) 
    break

$LPI - About Face At Laredo Petroleum. https://t.co/07ip7XqPlZ #trading #finance #stockmarket


In [9]:
search_words = "#wildfires"
date_since= "2018-11-16"  
# Collect tweets
tweets = Cursor(api.search,
              q=search_words,
              lang="en", 
              since=date_since).items(2)
# Iterate and print tweets
for tweet in tweets:
    print(tweet.text)

Thanks for the share @deborahblum! Alum of your Investigative Journalism class with @wiswatchandy back in 2011. Sti… https://t.co/nHVgbsk8pK
RT @OSFM: Timing couldn't be better with this beautiful video from our friends at @KeepOregonGreen. Please take extra care to protect our a…


#### get user info

get user/ttweets info of list of handles

In [10]:
def get_user_tweets_info(handles):
    
    cols = ['id', 'name', 'screen_name', 'description', 
            'statuses_count', 'friends_count', 'followers_count', 
            'account_age_days', 'avg_daily_tweets', 'hashtags',
            'user_mentions','favorite_count', 'retweet_count',]
    
    # dataframe that would be returned at the end
    df = pd.DataFrame(columns=cols)
    #print(df)
    handle_data = []
    off_users = []
            
    if len(handles) > 0: 
        for handle in handles:
            value_list = []
            print("Getting data for " + handle)
            # this helps avoid Tweepy errors like suspended users or user not ound errors
            try:
                item = api.get_user(handle)
            except tweepy.TweepError as e:
                continue
            value_list+= item.id_str, item.name, item.screen_name,\
            item.description, item.statuses_count, item.friends_count, item.followers_count
            
            #get average daily tweets
            
            no_tweets = item.statuses_count
            account_created_date = item.created_at
            delta = datetime.utcnow() - account_created_date
            account_age_days = delta.days
            value_list.append(str(account_age_days))
            #print(str(account_age_days))
            if account_age_days > 0:
                   value_list.append(int(float(no_tweets)/float(account_age_days)))
                    
            hashtags = []
            mentions = []
            favorite_count =[]
            retweet_count=[] 
            tweets = [] 
            tweet_count = 0
            end_date = datetime.utcnow() - timedelta(days=30)
            

            for status in Cursor(api.user_timeline, id=handle,
                              #  trim_user=True, exclude_replies=True
                        ).items(500):
#                 tweets.append(status._json['text'])
#                 print(tweets)
#                 break
                
                #tweets.append(status['text'])
                tweet_count+= 1
                if hasattr(status, "entities"):
                    entities = status.entities

                # get hashtags
                if "hashtags" in entities:
                    for ent in entities["hashtags"]:
                        if ent is not None:
                            if "text" in ent:
                                hashtag = ent["text"]
                                if hashtag is not None:
                                    hashtags.append(hashtag)
                # get usermentions
                if "user_mentions" in entities:
                    for ent in entities["user_mentions"]:
                        if ent is not None:
                            if "screen_name" in ent:
                                name = ent["screen_name"]
                                if name is not None:
                                    mentions.append(name)

                # get retweets    
                if hasattr(status, "retweet_count"):
                    retweets = status.retweet_count
                    if retweets is not None:
                        retweet_count.append(retweets)
                        
                # favorite count     
                if hasattr(status, "favorite_count"):
                    likes = status.favorite_count 
                    if likes is not None:
                        favorite_count.append(likes)
                if status.created_at < end_date:
                    break
                    
            
            value_list.append(len(hashtags))
            value_list.append(len(mentions))
            value_list.append(sum(favorite_count))
            value_list.append(sum(retweet_count))
            handle_data.append(value_list)
            
            tweet_series = pd.Series(tweets)
            df = df.append(pd.DataFrame([value_list], columns=cols))
            df=pd.concat([df, pd.DataFrame(tweet_series)], axis=1)
            df = df.rename(columns={0:'tweet'})
            #print(df)
            break
    return df


#### get tweets of each user

In [43]:
class Twitter_User():
    def __init__(self,id,count=200):
        self.id = id
        self.count = count
        self.data = None
    def get_tweets(self):
        store_tweets = Cursor(api.user_timeline, screen_name=self.id).items(500)
        simple_list = []
        for status in store_tweets:
            array = [status._json["text"].strip(), status._json["favorite_count"], status._json["created_at"],status._json["retweet_count"],[h["text"] for h in status._json["entities"]["hashtags"]]]
            simple_list.append(array)
        self.data = pd.DataFrame(simple_list, columns=["Text", "Like", "Created at","Retweet","Hashtags"])
        self.data = self.data[~self.data["Text"].str.startswith('RT')]
        return self.data
    def __repr__(self):
        id = api.get_user(self.id)
        return id.screen_name

In [44]:
 X= get_tweets(ls)

TweepError: [{'code': 34, 'message': 'Sorry, that page does not exist.'}]

In [ ]:
ls = []
for status in Cursor(api.user_timeline, screen_name='@realDonaldTrump', tweet_mode="extended").items(500):
    ls.append(status.full_text)

In [ ]:
don = Twitter_User('@realDonaldTrump', count=500)
don

In [42]:
don.get_tweets()

,Text,Like,Created at,Retweet,Hashtags
1,So Obama and his team of lowlifes spied on my ...,139768,Fri Jul 24 23:38:05 +0000 2020,39505,[]
2,96% Approval Rating in the Republican Party. T...,113269,Fri Jul 24 22:57:57 +0000 2020,18158,[]
10,Congratulations @JimRyun! https://t.co/I4iwvYx...,29522,Fri Jul 24 18:47:00 +0000 2020,7646,[]
11,I spoke to highly respected (Chairman) Senator...,115924,Fri Jul 24 14:28:39 +0000 2020,27586,[]
12,"Great job by @elonmusk in agreeing to build, i...",254891,Fri Jul 24 13:59:35 +0000 2020,43614,[]
13,The great Jim Ryun will be honored this mornin...,61806,Fri Jul 24 12:55:06 +0000 2020,13105,[]
14,https://t.co/gPAHGxgLKN,141872,Fri Jul 24 00:36:59 +0000 2020,31529,[]
15,Will be interviewed by @seanhannity tonight at...,81515,Fri Jul 24 00:18:50 +0000 2020,14144,[]
16,WELCOME BACK BASEBALL! https://t.co/dhLSfNkI7L,164765,Thu Jul 23 23:43:11 +0000 2020,29122,[]
18,White House Press Conference at 5:15 P.M. Live!,90906,Thu Jul 23 20:53:13 +0000 2020,13811,[]
